# Imports 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from numpy import pi, cos, sin, log, exp
from scipy.interpolate import interp1d
import numpy as np
import os.path
from datetime import datetime, timezone
import pyproj as proj
import cartopy
import cartopy.crs as ccrs

In [2]:
# Method for creating EPSG projection strings
# This is provided as an example 
# If the data spans one zone return a string representing the EPSG projection code
# If the data spans more than one zone return a list with as its first element the central zone as defined by
# the range of the data, and as its other members the other zones in which data is contained


def get_EPSG_projection_code(latitudes, longitudes, projection, ellipsoid):

    # By default return None
    epsg_codes = None

    if not isinstance(projection, str):
        raise RuntimeError(
            'get_EPSG_projection_code(): argument `projection` must be of type str')
  
    if not isinstance(ellipsoid, str):
        raise RuntimeError(
            'get_EPSG_projection_code(): argument `ellipsoid` must be of type str')

    if projection == 'UTC':

        if ellipsoid != 'WGS84':
            raise RuntimeError('get_EPSG_projection_code(): Currently the ellipsoid ' +
                           str(ellipsoid)+' is not yet implemented')
            
        else:
            ell_str='32'

        if isinstance(latitudes, float) and isinstance(longitudes, float):

            # Deal with the zones
            zone = str(int(np.floor((longitudes + 180) / 6) % 60 + 1))

            if len(zone) == 1:
                zone = '0'+zone

            # Deal with the hemispheres
            if latitudes >= 0:
                epsg_codes = ell_str+'6'+zone
            else:
                epsg_codes  =ell_str+'7'+zone

        elif isinstance(latitudes, list) and isinstance(longitudes, list):

            epsg_codes = list()

            # Determine the central longitude and latitude
            central_lon = (max(longitudes)+min(longitudes))/2
            central_lat = (max(latitudes)+min(latitudes))/2

            # Determine the UTM Zone from the central longitude
            zone = str(int((np.floor((central_lon + 180) / 6) % 60) + 1))

            if len(zone) == 1:
                zone = '0'+zone

            # Deal with the hemispheres
            if central_lat >= 0:
                zone = ell_str+'6'+zone
            else:
                zone = ell_str+'7'+zone

            epsg_codes.append(zone)

            ii = -1
            # Determine the UTM zone for all points in the lists
            for longitude in longitudes:

                # Update the index
                ii += 1

                zone = str(int(np.floor((longitude + 180) / 6) % 60 + 1))

                if len(zone) == 1:
                    zone = '0'+zone

                # Deal with the hemispheres
                if latitudes[ii] >= 0:
                    zone = ell_str+'6'+zone
                else:
                    zone = ell_str+'7'+zone

                if zone not in epsg_codes:
                    epsg_codes.append(zone)

            if len(epsg_codes) != 1:
                print('get_EPSG_projection_code(): Data spans more than one UTM zone!')

        else:
            raise RuntimeError(
                'get_EPSG_projection_code(): latitudes and longitudes must be both of type `float` or `list`')

    else:
        raise RuntimeError('get_EPSG_projection_code(): Projection `' +
                           projection+'` not yet implemented')

    return epsg_codes

# 1.1.1 Water Level Class

In [3]:
class WaterLevel:
    """A Class for handling Water Level Data"""

    def __init__(self):

        # The data attributes
        self.times = list()
        self.water_levels = list()
        self.metadata = dict()
        self.metadata["units"] = "m"
        self.metadata["geoid"] = None
        self.metadata["ellipsoid"] = None
        self.metadata["chart_datum"] = None
        self.metadata["start_time"] = None
        self.metadata["end_time"] = None
        self.metadata["count"] = None
        self.metadata["time_basis"] = "UTC"

    # The I/O methods:

    def read_jhc_file(self, fullpath):

        # Check the File's existence
        if os.path.exists(fullpath):
            self.metadata["Source File"] = fullpath
            print('Opening water level data file:' + fullpath)
            print(type(wl_lines))
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)

        # Open, read and close the file
        wl_file = open(fullpath)
        wl_content = wl_file.read()
        wl_file.close

        # Tokenize the contents
        wl_lines = wl_content.splitlines()
        count = 0  # initialize the counter for the number of rows read
        for wl_line in wl_lines:
            observations = wl_line.split()  # Tokenize the string
            epoch=datetime.fromtimestamp(float(observations[5]), timezone.utc)
            self.times.append(epoch)
            self.water_levels.append(float(observations[6]))
            count += 1
        print(type(wl_lines))
        
    def draw(self):
        plt.figure(figsize=(10, 10))
        print('Drawing Water Level Data')
  
        # plotting the points  
        plt.plot(self.times, self.water_levels) 
        plt.title('Water Levels in [m]') 
        plt.ylabel('Water Level in [m] →') 
        plt.xlabel('Time ('+self.metadata['time_basis']+') →') 
        plt.xticks(rotation='60')

  
        # giving a title to my graph 
        
  
        # function to show the plot 
        #plt.show() 

## 1.2 Class Definition for Position Data 

In [4]:
class Position:
    """A Class for handling Position Data"""

    def __init__(self):

        # The data attributes
        self.times = list()
        
        # The geodetic coordinates - these are curvilinear so do not put them
        # in vectors, as that is a linear concept
        self.latitudes = list()
        self.longitudes = list()
        self.ortho_heights = list()
        self.proj_pos=np.array([])
        self.metadata = dict()
        self.data_path=str()
        self.metadata["units"] = "m"
        self.metadata["geoid"] = None
        self.metadata["ellipsoid"] = None
        self.metadata["chart_datum"] = None
        self.metadata["start_time"] = None
        self.metadata["end_time"] = None
        self.metadata["count"] = None
        self.metadata["time_basis"] = "UTC"
        self.metadata["proj_str"] = None

    # The I/O methods:

    def read_jhc_file(self, fullpath):

        # Set the reference ellipsoid to WGS84

        self.metadata["ellipsoid"] = "WGS84"

        # Check the File's existence
        if os.path.exists(fullpath):
            self.data_path = fullpath
            print('Opening GNSS data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)

        # Open, read and close the file
        gnss_file = open(fullpath)
        gnss_content = gnss_file.read()
        gnss_file.close
        
        times=list();

        # Tokenize the contents
        gnss_lines = gnss_content.splitlines()
        count = 0  # initialize the counter for the number of rows read
        for gnss_line in gnss_lines:
            observations = gnss_line.split()  # Tokenize the string
            time = datetime.fromtimestamp(
                float(observations[5]), timezone.utc)
            times.append(time)
            self.latitudes.append(float(observations[8]))
            self.longitudes.append(float(observations[7]))
            self.ortho_heights.append(float(observations[6]))
            count += 1

        self.times=np.asarray(times)
        
        # Set the metadata
        
        self.metadata["chart_datum"] = 'Orthometric'

    def carto_project(self, projection_name, z_reference):

        # start by creating a projection parameter string following PROJ protocol

        if not isinstance(projection_name, str):
            raise RuntimeError(
                'Position.project(): argument `projection` must be of type str')

        if self.metadata["ellipsoid"] == None:
            raise RuntimeError(
                'Position.project(): Requires ellipsoid metadata to be defined!')

        # Keep a list of projection that are implemented in this function
        implemented_projections = list()
        implemented_projections.append('utm')

        # Raise an error of a non implemented projection is asked for
        if projection_name.lower() not in implemented_projections:
            raise RuntimeError(
                'Position.project(): The projection `' + projection_name + '` is not yet implemented')
            
        # Universal will map to utm or ups depending on the central latitude
        if projection_name.lower() == 'universal':
            projection_name='utm'

        if projection_name.lower() == 'utm':
            proj_str = '+proj=utm'

            # Determine the central longitude and latitude in degrees
            central_lon = (max(self.longitudes)+min(self.longitudes))/2
            central_lat = (max(self.latitudes)+min(self.latitudes))/2

            # Determine the UTM Zone from the central longitude and latitude
            proj_str += ' +zone=' + \
                str(int((np.floor((central_lon + 180) / 6) % 60) + 1))
            if central_lat > 0:
                proj_str += ' +north'
            else:
                proj_str += ' +south'

            # The geodetic datum of the input coordinates
            proj_str += ' +ellps=' + self.metadata["ellipsoid"]

            # The datum for the output coordinates
            proj_str += ' +datum=' + self.metadata["ellipsoid"]

            # The units for the output coordinates
            proj_str += ' +units=m'

            # Prevent PROJ default behavior
            proj_str += ' +no_defs'

            # Create a pyproj object using proj_str
            proj_obj = proj.Proj(proj_str)

            # Perform the projection
            E, N = proj_obj(self.longitudes, self.latitudes)
            
            # Create a matrix of positions as 3D column vectors
            if z_reference.lower()=='ortho':
                pos.proj_pos=np.asarray([np.asarray(E),np.asarray(N),np.asarray(self.ortho_heights)])
            else:
                raise RuntimeError('Position.carto_project(): currently only implemented for orthometric heights')
            
            # Add the string to the metadata
            self.metadata['proj_str'] = proj_str
            
    def get_carto(self):
        prj=''
        zone=''
        hemi=''
        ellipse=''
        datum=''
        s_carto=self.metadata['proj_str']
        for s in s_carto.split():
            s=s.split('=')
            if s[0] == '+proj':
                prj=s[1]
                print('he;;')
            elif s[0] == '+zone':
                zone=s[1]
            elif s[0] == '+north':
                 hemi = 'N'
            elif s[0] == '+south':
                 hemi = 'S'
            elif s[0] == '+ellps':
                 ellipse = s[1]
            elif s[0] == '+datum':
                 datum = s[1]
           
        if prj=='utm':
            return 'UTM zone ' + zone + hemi + ' on ' + ellipse

            
    def draw(self, projection='auto'):

        print('Drawing Positioning Data')

        # Dertermine the central latitude nand longitude
        central_lat = (max(self.latitudes)+min(self.latitudes))/2
        central_lon = (max(self.longitudes)+min(self.longitudes))/2

        # Create a 18x6 figure object
        fig = plt.figure(figsize=(18, 6))

        # Add a supertitle for the figure
        fig.suptitle("Positioning Data")

        # Create an Orthographic coordinate reference system (crs)
        crs_ortho = ccrs.Orthographic(central_lon, central_lat)

        # Plot the globe using the just defined crs in the first plot in a row of two subplots
        ax1 = fig.add_subplot(
            1, 2, 1, projection=crs_ortho)
        ax1.set_global()

        # Add Oceans, land and graticule
        ax1.add_feature(cartopy.feature.OCEAN)
        ax1.add_feature(cartopy.feature.LAND, edgecolor='black')
        ax1.gridlines()

        # Set the title
        ax1.set_title('Orthographic Map of Coverage Area')

        # Indicate the general area of the Positions by plotting a black marker with white edges
        # at the central latitude and longitude
        plt.plot(central_lon, central_lat, marker='o', markersize=7.0, markeredgewidth=2.5,
                 markerfacecolor='black', markeredgecolor='white',
                 transform=crs_ortho)

        # Set the zone number and hemisphere for the UTM projection
        zone_number = int((np.floor((central_lon + 180) / 6) % 60) + 1)
        if central_lat < 0:
            southern_hemisphere = True
        else:
            southern_hemisphere = False

        # Create an UTM reference system using the zone_number and hemisphere derived from the central coordinate
        crs_utm = ccrs.UTM(
            zone=zone_number, southern_hemisphere=southern_hemisphere)

        # Plot the zone using the UTM crs in the second plot of the two subplots
        ax2 = fig.add_subplot(
            1, 2, 2, projection=crs_utm)
        
        # Set display limits based on the extend of the geodetic coordinates in this object
        e_buffer=(np.max(self.longitudes)-np.min(self.longitudes))/10
        n_buffer=(np.max(self.latitudes)-np.min(self.latitudes))/5
        ax2.set_extent((np.min(self.longitudes)-e_buffer, np.max(self.longitudes)+e_buffer, np.min(
            self.latitudes)-n_buffer, np.max(self.latitudes)+n_buffer), crs=crs_utm)

        # Add Oceans, land and graticule
        ax2.add_feature(cartopy.feature.OCEAN, zorder=0)
        ax2.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
        ax2.gridlines()
        
        # Set the title 
        ax2.set_title('UTM Map of Positioning Data')


        # Plot all the positions as black dots
        for lon, lat in zip(self.longitudes, self.latitudes):
            plt.plot(lon, lat, marker='.', markersize=2.0,
                     markerfacecolor='black', transform=crs_utm)

        # Display the figure
        plt.show()

## 1.3 Class Definition for TWTT Data

In [5]:
class EchoSounderData:
    """A Class for handling Two Way Travel Time Data"""

    def __init__(self):

        # The data attributes
        self.times = np.array([])
        self.twtts = np.array([])
        self.metadata = dict()
        self.metadata["units"] = "s"
        self.metadata["start_time"] = None
        self.metadata["end_time"] = None
        self.metadata["count"] = None
        self.metadata["time_basis"] = "UTC"
        self.metadata["Source_File"]=str()
        
    # The I/O methods:

    def read_jhc_file(self, fullpath):

        # Check the File's existence
        if os.path.exists(fullpath):
            self.metadata["Source File"] = fullpath
            print('Opening Two Way Travel Time (TWTT) data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)

        # Open, read and close the file
        twtt_file = open(fullpath)
        twtt_content = twtt_file.read()
        twtt_file.close
        
        times=list()
        twtts=list()
        
        # Tokenize the contents
        twtt_lines = twtt_content.splitlines()
        count = 0  # initialize the counter for the number of rows read
        for twtt_line in twtt_lines:
            observations = twtt_line.split()  # Tokenize the string
            #time=datetime.fromtimestamp(float(observations[5]), timezone.utc)
            times.append(datetime.fromtimestamp(float(observations[5]), timezone.utc))
            twtts.append(float(observations[6]))
            count += 1
            
        self.times=np.asarray(times)
        self.twtts=np.array(twtts)
     
    def draw(self):
        plt.figure(figsize=(10, 10))
        print('Drawing TWTT Data')
     # plotting the points  
        plt.plot(self.times, self.twtts) 
        plt.title('Two Way Travel Times in [s]') 
        plt.ylabel('TWTT in [s] →') 
        plt.xlabel('Time time ('+self.metadata['time_basis']+') →') 
        plt.xticks(rotation='60')

## 1.4 Class Definition for Motion Data

In [6]:
class Motion:
    """A Class for handling motion Data"""

    def __init__(self):

        # The data attributes
        self.times = list()
        self.yaw = list()
        self.roll = list()
        self.pitch = list()
        self.heave = list()
        self.metadata = dict()
        self.metadata["units"] = "rad"
        self.metadata["start_time"] = None
        self.metadata["end_time"] = None
        self.metadata["count"] = None
        self.metadata["time_basis"] = "UTC"

    # The I/O methods:

    def read_jhc_file(self, fullpath):

        # Check the File's existence
        if os.path.exists(fullpath):
            self.metadata["Source File"] = fullpath
            print('Opening motion data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)

        # Open, read and close the file
        motion_file = open(fullpath)
        motion_content = motion_file.read()
        motion_file.close

        # Tokenize the contents
        motion_lines = motion_content.splitlines()
        count = 0  # initialize the counter for the number of rows read
        for motion_line in motion_lines:
            observations = motion_line.split()  # Tokenize the string
            time = datetime.fromtimestamp(
                float(observations[5]), timezone.utc)
            self.times.append(time)
            self.yaw.append(float(observations[6])*pi/180)
            self.roll.append(float(observations[7])*pi/180)
            self.pitch.append(float(observations[8])*pi/180)
            self.heave.append(float(observations[9]))
            count += 1

    def draw(self):
        print('Drawing Motion Data')
        plt.figure(figsize=(20, 10))
        ax1 = plt.subplot(4, 1, 1)
        plt.plot(self.times, np.degrees(self.yaw))
        plt.ylabel('Heading [deg] →')
        ax2 = plt.subplot(4, 1, 2, sharex=ax1)
        plt.plot(self.times, self.heave)
        plt.ylabel('Heave [m] →')
        ax3 = plt.subplot(4, 1, 3, sharex=ax1)
        plt.plot(self.times, np.degrees(self.roll))
        plt.ylabel('Roll [deg] →')
        ax4 = plt.subplot(4, 1, 4, sharex=ax1, sharey=ax3)
        plt.plot(self.times, np.degrees(self.pitch))
        plt.ylabel('Pitch [deg] →')
        plt.xlabel('Time ('+self.metadata['time_basis']+') →')
        plt.xticks(rotation='60')

        plt.setp(ax1.get_xticklabels(), visible=False)
        plt.setp(ax2.get_xticklabels(), visible=False)
        plt.setp(ax3.get_xticklabels(), visible=False)

## 2.1 Class Definition for Sound Speed Profile Data

In [7]:
class SSP:
    """A Class for handling Sound Speed Profile data"""

    def __init__(self):

        # The data attributes
        self.obs_time = None
        self.log_time = None
        self.obs_latitude = None
        self.obs_longitude = None
        self.vessel_latitude = None
        self.vessel_longitude = None
        self.obs_sample = list()
        self.obs_depth = list()
        self.obs_ss = list()
        self.proc_ss = np.array([])
        self.twtt_layer=np.array([])

        self.metadata = dict()
        self.metadata["units"] = "rad"
        self.metadata["count"] = None
        self.metadata["geoid"] = None
        self.metadata["ellipsoid"] = None
        self.metadata["chart_datum"] = None
        self.metadata["time_basis"] = "UTC"

    # The I/O methods:

    def read_jhc_file(self, fullpath):

        # Check the File's existence
        if os.path.exists(fullpath):
            self.metadata["Source File"] = fullpath
            print('Opening sound speed profile data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)

        # Open, read and close the file
        motion_file = open(fullpath)
        motion_content = motion_file.read()
        motion_file.close

        # Tokenize the contents
        motion_lines = motion_content.splitlines()
        self.obs_time = datetime.fromtimestamp(
            float(motion_lines[1].split()[0]), timezone.utc)
        self.log_time = datetime.fromtimestamp(
            float(motion_lines[2].split()[0]), timezone.utc)
        self.obs_latitude = float(motion_lines[3].split()[0])
        self.obs_longitude = float(motion_lines[3].split()[1])
        self.vessel_latitude = float(motion_lines[4].split()[0])
        self.vessel_longitude = float(motion_lines[4].split()[1])
        self.metadata["count"] = int(motion_lines[5].split()[0])

        count = 0  # initialize the counter for the number of rows read

        for motion_line in motion_lines[16:]:
            observations = motion_line.split()  # Tokenize the stringS
            self.obs_sample.append(float(observations[0]))
            self.obs_depth.append(float(observations[1]))
            self.obs_ss.append(float(observations[2]))
            count += 1

        if self.metadata["count"] != count:
            raise RuntimeError('Nr of Samples read ('+str(count) +
                               ') does not match metadata count (' +
                               str(self.metadata["count"])+')')

        # Process the data - in the jhc data files this is already a one-way profile,
        # this just for illustration
        self.proc_ss = np.zeros((count, 3))

        # Sort the data samples by depth
        sorted_ss = sorted(zip(self.obs_depth, self.obs_ss))

        layer = 0
        for d, ss in sorted_ss:
            self.proc_ss[[layer], [0]] = d
            self.proc_ss[[layer], [1]] = ss
            layer += 1

        # Identify all the depths for which there are multiple observations
        mask = np.full((count, 1), True)
        mask[1:, [0]] = np.diff(self.proc_ss[:, [0]], axis=0) != 0

        # Remove the duplicates - You really should get statistical representations here
        # but to keep this short just remove the duplicates
        self.proc_ss = self.proc_ss[mask[:, 0], ...]

        # Determine the gradients - Note the indexing: the gradient of the first layer 
        # is contained at the same index as the data for the TOP of the layer.
        self.proc_ss[0:-1, [2]] = np.diff(self.proc_ss[:, [1]],
                                          axis=0)/np.diff(self.proc_ss[:, [0]], axis=0)

        # Estimate gradient for last layer assuming that the temperature and salinity remain the same
        # gradient solely a function of pressure (depth)
        self.proc_ss[-1, [2]] = 0.017

        # Extend to 12000 m if necesarry - this is to get around some manufcturers requirements
        if self.obs_depth[-1] < 12000:
            ss = self.proc_ss[-1:, [1]] + self.proc_ss[-1:, [2]] \
             * (12000-self.proc_ss[-1:, [0]])
            self.proc_ss = np.vstack((self.proc_ss, [12000, ss, 0.017]))

        # Extend to 0 m if necesarry - assume well mixed
        if self.obs_depth[0] > 0:
            self.proc_ss = np.vstack(
                ([0, self.proc_ss[0, [1]], 0.], self.proc_ss))
            
        # Step 5 Create a look-up array of twtts for each full layer
        # Allows for great gain in efficiency (do not have to calculate for each ping)
        self.twtt_layer = np.zeros((count, 1))
        
        for layer in range(0,self.metadata["count"]-1):
            if self.proc_ss[layer, [2]] == 0:
                self.twtt_layer[layer] = 2 * \
                    (self.proc_ss[layer+1, [0]] - self.proc_ss[layer, [0]])/ \
                     self.proc_ss[layer, [1]]
            else:
                self.twtt_layer[layer] = 2 / self.proc_ss[layer, [2]] * \
                 log(self.proc_ss[layer+1, [1]]/self.proc_ss[layer, [1]])

    def depth(self, start_depth, twtt):
        # Determine depth relative to the transducer for vertical incidence data 
        # using the harmonic mean sound speed

        # Find the index to the start layer using Boolean logic
        # Note that the profile is extended to full ocean depth
        # As layer is indexed by the top depth this is a guarantee that there is a next layer
        start_i = sum(start_depth >= self.proc_ss[:, [0]])-1
        start_i = int(start_i[0])  # Turn the index into an integer

        # The sound speed at the top of the layer
        c_start=self.proc_ss[start_i, [1]]
        
        # Calculate the time it takes to get through this layer - avoid division by zero
        if self.proc_ss[start_i, [2]] == 0:
            twtt_layer = 2*(self.proc_ss[start_i+1, [0]] - \
                start_depth)/self.proc_ss[start_i, [1]]
        else:
            # Update c_start to comepensate for the change in speed to the transducer depth
            c_start += self.proc_ss[start_i, [2]]*(start_depth-self.proc_ss[start_i, [0]])
            
            twtt_layer = 2/self.proc_ss[start_i, [2]] * \
             log(self.proc_ss[start_i+1, [1]]/c_start)
            
        # Create a cumulative sum of the contributions of each layer
        twtt_cum=np.zeros((self.metadata["count"]))
        twtt_cum[start_i+1]=twtt_layer
        twtt_cum[start_i+2:]=twtt_layer+np.cumsum(self.twtt_layer[start_i+2:])
        
        # Determine where twtt_cum starts to exceed twtt using Boolean logic
        end_i=np.max(np.where(twtt_cum<twtt),axis=1)

        # Deal with the twtt in the final layer. There are two cases 
        # 1) the signal originated in a higher layer 
        # 2) the signal originated in the same layer  
        
        if start_i!=end_i:
            twtt_last=(twtt-twtt_cum[end_i])
            d_start=self.proc_ss[end_i, [0]]
        else: 
            twtt_last=twtt
            d_start=start_depth
            
        if  self.proc_ss[[end_i], [2]]!=0:
            # gradient is non-zero
            # From twtt=2/g*log(C2/C1) => C2=C1*exp(g*twtt/2)
            c_end = c_start*exp( self.proc_ss[end_i, [2]]*twtt_last/2)
            
            # The associated depth
            depth = d_start + (c_end-c_start)/self.proc_ss[end_i, [2]]
            
        else:
            # gradient is zero
            depth=d_start + self.proc_ss[end_i, [1]]*twtt_last/2

        # the depth is relative to the instantaneous (heaving) water surface, 
        # bring the value back to the transducer.
        
        depth-=start_depth   
        return depth

    def draw(self, full_profile=False):
        print('Drawing Sound Speed Data')
        plt.figure(figsize=(12, 6))

        ax1 = plt.subplot(1, 3, 1)
        if full_profile:
            plt.plot(self.obs_ss[0:], self.obs_depth[0:])
        else:
            plt.plot(self.obs_ss[0:-1], self.obs_depth[0:-1])

        plt.ylabel('← Depth [m]')
        plt.xlabel('Sound Speed [m/s] →')
        ax1.invert_yaxis()
        ax1.xaxis.tick_top()
        ax1.xaxis.set_label_position('top')

        ax2 = plt.subplot(1, 3, 2)
        if full_profile:
            plt.plot(self.proc_ss[0:, [2]], self.proc_ss[0:, [0]])
        else:
            plt.plot(self.proc_ss[0:-1, [2]], self.proc_ss[0:-1, [0]])

        plt.ylabel('← Depth [m]')
        plt.xlabel('Sound Speed Gradient [1/s] →')
        ax2.invert_yaxis()
        ax2.xaxis.tick_top()
        ax2.xaxis.set_label_position('top')
        plt.show()

In [8]:
class Vessel:
    """A Class for handling Vessel Specific Data"""

    def __init__(self):

        # The metadata
        self.metadata = dict()
        self.metadata["name"]=str()
        self.metadata["ownedby"]=str()
        self.metadata["operatedby"]=str()
    
        # Quantitative data
        self.lever_arm_trans =  np.array([])
        self.lever_arm_rec =  np.array([])
        self.lever_arm_pos = np.array([])
        self.lever_arm_mru = np.array([])        
        wl=[]
        

In [9]:
class Integration:
    """A Class for Integrating Data to Create Soundings"""

    def __init__(self, twtt, pos, motions, sound_speed_profile, water_levels, vessel):
        
        # For now we can only integrate if the Positions have been projected - we will use 
        # either UTM or UPS depending on the latitude
        
        if not pos.proj_pos.any():
            pos.carto_project('universal')
        
        # The variables passed in
        self.twtt=twtt
        self.pos_ant=pos
        self.motions=motions
        self.ssp=sound_speed_profile
        self.water_levels=water_levels
        self.vessel=vessel
        
        # The number of twtts
        n_twtt_times = len(twtt.times)

        # the variables determined in the integration
        self.R_tx = list()
        self.R_rx = list()
        self.lever_arm_pos_tx=np.zeros([3,n_twtt_times])
        self.lever_arm_pos_rx=np.zeros([3,n_twtt_times])
        self.lever_arm_trans_tx=np.zeros([3,n_twtt_times])
        self.lever_arm_rec_rx=np.zeros([3,n_twtt_times])
        self.pos_rp_tx=np.zeros([3,n_twtt_times])
        self.pos_rp_rx=np.zeros([3,n_twtt_times])
        self.pos_trans_tx=np.zeros([3,n_twtt_times])
        self.pos_rec_rx=np.zeros([3,n_twtt_times])
              

        # Determine the transmit times as posix times
        t_twtt = np.array([e.timestamp() for e in twtt.times])

        # Obtaine the times of the various observed data as posix times
        t_pos = np.array([e.timestamp() for e in pos.times])
        t_mru = np.array([e.timestamp() for e in motions.times])
        t_wl = np.array([e.timestamp() for e in water_levels.times])
        
        # Determine the interpolation function for the positions
        f=interp1d(t_pos,pos.proj_pos,bounds_error=False)
        
        # Interpolate for the time of transmit
        # Offset the heading by pi/2 to align axes
        self.pos_proj_ant_tx=f(t_twtt)
        self.p_tx = np.interp(t_twtt, t_mru, motions.pitch)
        self.r_tx = np.interp(t_twtt, t_mru, motions.roll)
        self.y_tx = np.interp(t_twtt, t_mru, motions.yaw)
        self.h_tx = np.interp(t_twtt, t_mru, motions.heave)
        self.wl_tx = np.interp(t_twtt, t_wl, water_levels.water_levels)

        # Determine the reception times as posix times
        t_twtt += twtt.twtts

        # Interpolate for the time of reception
        self.pos_proj_ant_rx=f(t_twtt)
        self.p_rx = np.interp(t_twtt, t_mru, motions.pitch)
        self.r_rx = np.interp(t_twtt, t_mru, motions.roll)
        self.y_rx = np.interp(t_twtt, t_mru, motions.yaw)
        self.h_rx = np.interp(t_twtt, t_mru, motions.heave)
        self.wl_rx = np.interp(t_twtt, t_wl, water_levels.water_levels)

        # For each ping (twtt) determine a sounding solution
        ping = 0
        self.depth = np.zeros((n_twtt_times))
        self.la_trans_rec_txrx = np.zeros((3,n_twtt_times))


        for t in t_twtt:

            # Calculate the right-handed Euclidean Euler andgle Rotation matrices at the
            # time of transmit around the x,y and z axes
            Rx_tx = np.array([[1, 0,                     0                   ],
                              [0, cos(self.r_tx[ping]), -sin(self.r_tx[ping])],
                              [0, sin(self.r_tx[ping]),  cos(self.r_tx[ping])]])

            Ry_tx = np.array([[cos(self.p_tx[ping]),  0, sin(self.p_tx[ping]) ],
                              [0,                     1,  0                   ],
                              [-sin(self.p_tx[ping]), 0,  cos(self.p_tx[ping])]])

            Rz_tx = np.array([[cos(self.y_tx[ping]), -sin(self.y_tx[ping]), 0],
                              [sin(self.y_tx[ping]),  cos(self.y_tx[ping]), 0],
                              [0,                     0,                    1]])

            # Calculate the total rotation matrix at transmit in the order x, y, z
            self.R_tx.append( Rz_tx@Ry_tx@Rx_tx)

            # Rotation matrices at time of reception
            Rx_rx = np.array([[1, 0,                     0                   ],
                              [0, cos(self.r_rx[ping]), -sin(self.r_rx[ping])],
                              [0, sin(self.r_rx[ping]),  cos(self.r_rx[ping])]])

            Ry_rx = np.array([[cos(self.p_rx[ping]),  0,  sin(self.p_rx[ping])],
                              [0,                     1,  0                   ],
                              [-sin(self.p_rx[ping]), 0,  cos(self.p_rx[ping])]])

            Rz_rx = np.array([[cos(self.y_rx[ping]), -sin(self.y_rx[ping]), 0],
                              [sin(self.y_rx[ping]),  cos(self.y_rx[ping]), 0],
                              [0,                     0,                    1]])

            # Calculate the total rotation matrix at receive in the order x, y, z
            self.R_rx.append( Rz_rx@Ry_rx@Rx_rx)

            # Calculate the georeferenced lever_arms at the transmit times
            # i.e., rotate the vessel lever arms using the rotation matrix R_tx
            self.lever_arm_pos_tx[:,[ping]]=self.R_tx[ping]@vessel.lever_arm_pos
            self.lever_arm_trans_tx[:,[ping]]=self.R_tx[ping]@vessel.lever_arm_trans

            # Calculate the geo referenced lever_arms at the reception times using R_rx
            self.lever_arm_pos_rx[:,[ping]]=self.R_rx[ping]@vessel.lever_arm_pos
            self.lever_arm_rec_rx[:,[ping]]=self.R_rx[ping]@vessel.lever_arm_rec
            
            # Calculate the lever arm to the virtual transucer located at the mean position
            # of the transmitter and receiver
            self.la_trans_rec_txrx[:,[ping]] = (self.lever_arm_trans_tx[:,[ping]]+self.lever_arm_rec_rx[:,[ping]])/2

            # Calculate depth as height relative to the ships transducer
            # Note that this requires both the waterline and the lever_arm to get the right starting point 
            # in the water column - not heave - as we assume that the profile moves up and down with the surface.
            self.depth[ping] = sound_speed_profile.depth(self.la_trans_rec_txrx[2,[ping]]-vessel.wl, twtt.twtts[ping])
            
            # Now determine the rp and transducer position at the transmits
            # Be careful about axis alignment!
            self.pos_rp_tx[0,[ping]]=self.pos_proj_ant_tx[0,[ping]]-self.lever_arm_pos_tx[1,[ping]]
            self.pos_rp_tx[1,[ping]]=self.pos_proj_ant_tx[1,[ping]]-self.lever_arm_pos_tx[0,[ping]]
            self.pos_rp_tx[2,[ping]]=self.pos_proj_ant_tx[2,[ping]]-self.lever_arm_pos_tx[2,[ping]]
            self.pos_trans_tx[:,[ping]]=self.pos_rp_tx[:,[ping]]+self.lever_arm_trans_tx[:,[ping]]

            # Now determine the rp and transducer position at reception
            self.pos_rp_rx[0,[ping]]=self.pos_proj_ant_rx[0,[ping]]-self.lever_arm_pos_rx[1,[ping]]
            self.pos_rp_rx[1,[ping]]=self.pos_proj_ant_rx[1,[ping]]-self.lever_arm_pos_rx[0,[ping]]
            self.pos_rp_rx[2,[ping]]=self.pos_proj_ant_rx[2,[ping]]-self.lever_arm_pos_rx[2,[ping]]
            self.pos_rec_rx[:,[ping]]=self.pos_rp_rx[:,[ping]]+self.lever_arm_rec_rx[:,[ping]]

            # update the ping count
            ping += 1
            
        print("Done integrating sounding data from source"+twtt.metadata['Source_File'])

    def draw(self):
        fig=plt.figure(figsize=(12, 6))
        ax1 = plt.subplot(3,1,1)
        plt.plot(self.twtt.times, self.twtt.twtts*np.mean(self.ssp.obs_ss)/2)
        plt.plot(self.twtt.times, self.twtt.twtts *
             np.mean(self.ssp.obs_ss)/2+(self.h_tx+self.h_rx)/2+self.la_trans_rec_txrx[2,:])
        plt.plot(self.twtt.times, self.depth+(self.h_tx+self.h_rx)/2+self.la_trans_rec_txrx[2,:])

        plt.title('Depths [m]')
        plt.ylabel('Depths [m] →')
        plt.xlabel('Time ('+self.twtt.metadata['time_basis']+') →')
        ax1.invert_yaxis()
    
        ax2 = plt.subplot(3,1,2)
        plt.plot(self.twtt.times, (self.twtt.twtts * \
         np.mean(self.ssp.obs_ss)/2+(self.h_tx+self.h_rx)/2+self.la_trans_rec_txrx[2,:])-(self.depth+(self.h_tx+self.h_rx)/2+self.la_trans_rec_txrx[2,:]))
    
        plt.title('Depths [m]')
        plt.ylabel('Depths [m] →')
        plt.xlabel('Time ('+self.twtt.metadata['time_basis']+') →')
        ax2.invert_yaxis()
        
        ax3 = plt.subplot(3,1,3)
        twtt_times=self.twtt.times[0:100]
        twtts=self.twtt.twtts[0:100]
        obs_ss=self.ssp.obs_ss[0:100]
        h_tx=self.h_tx[0:100]
        h_rx=self.h_rx[0:100]
        la=self.la_trans_rec_txrx[2,0:100]
        depth=self.depth[0:100]
        ant_tx_full=self.pos_proj_ant_rx[:,0:100]
        rp_tx_full=self.pos_rp_tx[:,0:100]
        trans_tx_full=self.pos_trans_tx[:,0:100]
        
        plt.plot(twtt_times, (twtts * \
                  np.mean(obs_ss)/2+(h_tx+h_rx)/2+la)- \
                   (depth+(h_tx+h_rx)/2+la))
    
        plt.plot(twtt_times,(h_tx+h_rx)/300+.125)
        plt.title('Depths [m]')
        plt.ylabel('Depths [m] →')
        plt.xlabel('Time ('+self.twtt.metadata['time_basis']+') →')
        ax3.invert_yaxis()
        
        # Space out the plots so that they do not overlap
        plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=.5, wspace=0.4)
        plt.show()        
        
        # Plot the navigation
        fig=plt.figure(figsize=(12, 6))
        ax4 = plt.subplot(2,1,1)
        plt.plot(self.pos_proj_ant_rx[0,:],self.pos_proj_ant_rx[1,:],'b.',label='Antenna')
        plt.plot(self.pos_rp_tx[0,:],self.pos_rp_tx[1,:],'r.',label='RP')
        plt.plot(self.pos_trans_tx[0,:],self.pos_trans_tx[1,:],'k.',label='Transducer')
        plt.legend()
        plt.axis('equal')
        ax5 = plt.subplot(2,1,2)
        plt.plot(ant_tx_full[0,0:100],ant_tx_full[1,0:100],'b.',label='Antenna')
        plt.plot(rp_tx_full[0,0:100],rp_tx_full[1,0:100],'r.',label='RP')
        plt.plot(trans_tx_full[0,0:100],trans_tx_full[1,0:100],'k.',label='Transducer')
        plt.axis('equal')
        plt.legend()
        plt.show()
    
    def draw_depths(self):
        fig=plt.figure(figsize=(12, 6))
        plt.plot(self.twtt.times, self.depth+(self.h_tx+self.h_rx)/2+self.la_trans_rec_txrx[2,:])

        plt.title('Depths [m]')
        plt.ylabel('Depths [m] →')
        plt.xlabel('Time ('+self.twtt.metadata['time_basis']+') →')
        plt.gca().invert_yaxis()
    
            
    def heave_gnss():
        # Determine heave from the rp trajectorty
        # Use an 30 second filtering window
         return heave
        
    def qc(self):
        # The match of the IMU heave to the positioning z-component is one quality control indicator
        
        fig=plt.figure(figsize=(12, 6))
        ax1 = plt.subplot(3,1,1)
#         plt.plot(self.twtt.times, self.pos_ant
#         plt.plot(self.twtt.times, self.depth+(self.h_tx+self.h_rx)/2+self.la_trans_rec_txrx)
        

In [10]:
# Create soundings by integrating the various data streams.
# We also need a Vessel class to store the geometric data descriptive of the vessel

vessel = Vessel()
vessel.lever_arm_trans = np.array([16.26, -1.75,   4.15]).reshape((3, 1))
vessel.lever_arm_rec = np.array([14.82, -2.01,   4.17]).reshape((3, 1))
vessel.lever_arm_pos = np.array([-5.73, -0.12, -30.00]).reshape((3, 1))
vessel.lever_arm_mru = np.array([0, 0, 0]).reshape((3, 1))
vessel.wl = -2.59

# Get the data path
abs_path = os.path.abspath(os.path.curdir)

# TWTTs
twtt = EchoSounderData()
twtt.read_jhc_file(abs_path+'/Lab_A_TWTT.txt')

# positions
pos = Position()
pos.read_jhc_file(abs_path+'/Lab_A_GNSS.txt')
# make sure that there is Cartesian representation of the positions, z-coordinates should be orthometric
pos.carto_project('utm','ortho')

# Motion data
motions = Motion()
motions.read_jhc_file(abs_path+'/Lab_A_MRU.txt')

# Water level data
water_levels = WaterLevel()
water_levels.read_jhc_file(abs_path+'/Lab_A_TIDE.txt')

# Sound speed data
sound_speed_profile = SSP()
sound_speed_profile.read_jhc_file(abs_path+'/Lab_A_SVP.txt')

# Integrate all the data
integration=Integration(twtt,pos,motions,sound_speed_profile, water_levels, vessel)


Opening Two Way Travel Time (TWTT) data file:/home/jupyter-semmed/ESCI_OE_774_874/Lab_A/Lab_A_TWTT.txt
Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_774_874/Lab_A/Lab_A_GNSS.txt
Opening motion data file:/home/jupyter-semmed/ESCI_OE_774_874/Lab_A/Lab_A_MRU.txt
Opening water level data file:/home/jupyter-semmed/ESCI_OE_774_874/Lab_A/Lab_A_TIDE.txt


UnboundLocalError: local variable 'wl_lines' referenced before assignment

In [ ]:
water_levels.draw()

***

Call the `draw` method of the object `pos` to draw the general location of the survey on the earth and a large scale plot of the navigation data

In [ ]:
pos.draw()

***

Show the associated motion data

In [ ]:
motions.draw()

***

Finally, show the two way travel data:

In [ ]:
twtt.draw()

In [ ]:
ping=0
# integration.la_trans_rec_txrx[:,[ping]] = (integration.lever_arm_trans_tx[:,[ping]]+integration.lever_arm_rec_rx[:,[ping]])/2
# integration.draw()
(integration.lever_arm_trans_tx[:,[ping]]+integration.lever_arm_rec_rx[:,[ping]])/2
integration.draw()
integration.draw_depths()


In [ ]:
ax=plt.subplot()
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter("%d"))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%d"))
plt.title('Positions Moving East (' + pos.get_carto()+')')
plt.plot(integration.pos_proj_ant_rx[0,0:100],integration.pos_proj_ant_rx[1,0:100],'b.',label='Antenna')
plt.plot(integration.pos_rp_rx[0,0:100],integration.pos_rp_rx[1,0:100],'r.',label='RP')
plt.plot(integration.pos_rec_rx[0,0:100],integration.pos_rec_rx[1,0:100],'k.',label='Tx Transducer')
plt.legend()
plt.show()

pos.proj_pos[:,[0]]

In [ ]:
lever_arm_pos_tx=np.zeros([3,100])

In [ ]:
def carto_print():
    pos.metadata['proj_str']
    s_carto=pos.metadata['proj_str']
    prj=''
    for ss in s_carto.split():
        ss=ss.split('=')
        if s[0] == '+proj':
            prj=s[1]
        elif s[0] == '+zone':
            zone=s[1]
        elif s[0] == '+north':
             hemi = 'N'
        elif s[0] == '+south':
             hemi = 'S'
        elif s[0] == '+ellps':
             ellipse = s[1]
        elif s[0] == '+datum':
             datum = s[1]
           
    if prj=='utm':
        print( 'UTM zone: ' + zone + hemi + ' on ' + ellipse)
         

In [ ]:
pos.metadata['proj_str']
s_carto=pos.metadata['proj_str']
prj=''
for ss in s_carto.split():
    ss=ss.split('=')
    if ss[0] == '+proj':
        prj=ss[1]
    elif ss[0] == '+zone':
        zone=ss[1]
    elif ss[0] == '+north':
        hemi = 'N'
    elif ss[0] == '+south':
        hemi = 'S'
    elif ss[0] == '+ellps':
        ellipse = ss[1]
    elif ss[0] == '+datum':
        datum = ss[1]
           
if prj=='utm':
    print( 'UTM zone: ' + zone + hemi + ' on ' + ellipse)
         

In [ ]:
pos.metadata['proj_str']